In [ ]:
import pickle
import torch
import pandas as pd
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine
from PIL import Image
import requests
import numpy as np
import cv2

# Initialize MTCNN and InceptionResnetV1
mtcnn = MTCNN(keep_all=True, thresholds=[0.5, 0.6, 0.6], min_face_size=15)  # Adjusted thresholds and size
model = InceptionResnetV1(pretrained='vggface2').eval()

# Load embeddings dataset
def load_embeddings(model_file):
    with open(model_file, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

# Compare embeddings with the dataset
def compare_faces(embedding, dataset, threshold=0.5):
    embedding = embedding / np.linalg.norm(embedding)  
    min_distance = float('inf')
    identified_name = None
    for name, embeddings in dataset.items():
        for known_embedding in embeddings:
            known_embedding = known_embedding / np.linalg.norm(known_embedding)  
            distance = cosine(embedding, known_embedding.flatten())
            if distance < min_distance:
                min_distance = distance
                identified_name = name
    if min_distance < threshold:
        return identified_name
    return None

# Load ID mapping
def load_id_mapping(excel_file):
    df = pd.read_excel(excel_file)
    return {row['Name']: row['ID'] for _, row in df.iterrows()}

# Download image from Google Drive
def download_image_from_url(image_url):
    file_id = image_url.split('id=')[-1]
    direct_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    response = requests.get(direct_url)
    if response.status_code != 200:
        raise Exception(f"Error downloading image. Status code: {response.status_code}")
    img_array = np.frombuffer(response.content, np.uint8)
    image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    if image is None:
        raise Exception("Failed to decode image. Check the URL or image format.")
    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# Recognize faces
def recognize_faces(image_url, model_file, excel_file, threshold=0.5):
    dataset = load_embeddings(model_file) 
    name_to_id = load_id_mapping(excel_file)

    try:
        image = download_image_from_url(image_url)
    except Exception as e:
        print(f"Error: {str(e)}")
        return

    faces, probs = mtcnn(image, return_prob=True)
    unknown_count = 0

    if faces is not None:
        for i, (face, prob) in enumerate(zip(faces, probs)):
            if prob < 0.90: 
                continue

            face = (face - 0.5) / 0.5  
            embedding = model(face.unsqueeze(0)).detach().cpu().numpy().flatten()
            name = compare_faces(embedding, dataset, threshold)

            if name:
                person_id = name_to_id.get(name, "ID not found")
                print(f"Student Name: {name}\nID: {person_id}")
            else:
                unknown_count += 1

        if unknown_count > 0:
            print(f"There {'is' if unknown_count == 1 else 'are'} {unknown_count} unknown face(s).")
    else:
        print("No faces detected.")

model_file = "face_model.pkl"
excel_file = "test.xlsx"
image_url = "https://drive.google.com/file/d/1Zjv1CmvyeW42m4haKxQXVcYFCIZXDqvW/view?usp=sharing"
recognize_faces(image_url, model_file, excel_file)
